In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.manifold import TSNE

In [ ]:
file_path2 = '20240612_family history.csv'
FH = pd.read_csv(file_path2)

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np
import pickle

# Load the dataset
file_path = '20240612_family history.csv'
df = pd.read_csv(file_path)

# Prepare results storage
analysis_results = {}

# Calculate global totals for affected across all ages for each question
total_affected_per_question = df.groupby('question').agg({
    'CeD_affected': 'sum',
    'healthy_affected': 'sum'
}).to_dict()

# Process each question
for question in df['question'].unique():
    df_filtered = df[df['question'] == question]
    contingency_tables = {}
    chi_square_results = {}
    
    # Total respondents (assuming the same for each age group, adjust if not)
    CeD_respondents_total = df_filtered['CeD_respondent'].iloc[0]
    healthy_respondents_total = df_filtered['healthy_respondants'].iloc[0]

    # Calculate total not affected
    CeD_not_affected_total = CeD_respondents_total - total_affected_per_question['CeD_affected'][question]
    healthy_not_affected_total = healthy_respondents_total - total_affected_per_question['healthy_affected'][question]

    # Process each age group within the filtered dataset
    for age_group, group_data in df_filtered.groupby('age_group'):
        contingency_table = [
            [group_data['CeD_affected'].sum(), CeD_not_affected_total],
            [group_data['healthy_affected'].sum(), healthy_not_affected_total]
        ]
        
        # Compute chi-square test and odds ratio
        chi2, p, dof, ex = chi2_contingency(contingency_table)
        a, b = contingency_table[0]
        c, d = contingency_table[1]
        odds_ratio = (a * d) / (b * c) if b * c != 0 else np.nan
        
        # Store results
        contingency_tables[age_group] = contingency_table
        chi_square_results[age_group] = {"chi2": chi2, "p-value": p, "odds_ratio": odds_ratio}
    
    # Store all results for the question
    analysis_results[question] = {
        "contingency_tables": contingency_tables,
        "chi_square_results": chi_square_results
    }

# Save results to a pickle file
with open('analysis_results.pkl', 'wb') as f:
    pickle.dump(analysis_results, f)

# Optional: Save contingency tables and additional metrics to a CSV for easy viewing
contingency_df = pd.DataFrame([
    (q, ag, *ct[0], *ct[1], res['p-value'], res['odds_ratio']) 
    for q, d in analysis_results.items() 
    for ag, ct, res in zip(d['contingency_tables'].keys(), d['contingency_tables'].values(), d['chi_square_results'].values())
], columns=['Question', 'Age Group', 'CeD Affected', 'CeD Not Affected', 'Healthy Affected', 'Healthy Not Affected', 'P-value', 'Odds Ratio'])

contingency_df.to_csv('contingency_tables.csv', index=False)





### try to calculate the contingency chisquare P value

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np
import pickle

# Load the dataset
file_path = '20240612_family history.csv'
df = pd.read_csv(file_path)

# Prepare results storage
analysis_results = {}

# Process each question
for question in df['question'].unique():
    df_filtered = df[df['question'] == question]
    contingency_tables = {}
    chi_square_results = {}

    # Calculate total affected and respondents
    total_CeD_affected = df_filtered['CeD_affected'].sum()
    total_healthy_affected = df_filtered['healthy_affected'].sum()
    total_CeD_respondents = df_filtered['CeD_respondent'].iloc[0]
    total_healthy_respondents = df_filtered['healthy_respondants'].iloc[0]

    # Calculate overall not affected
    total_CeD_not_affected = total_CeD_respondents - total_CeD_affected
    total_healthy_not_affected = total_healthy_respondents - total_healthy_affected

    # Overall contingency table
    overall_contingency_table = [
        [total_CeD_affected, total_CeD_not_affected],
        [total_healthy_affected, total_healthy_not_affected]
    ]
    
    # Compute overall chi-square test and odds ratio
    overall_chi2, overall_p, overall_dof, _ = chi2_contingency(overall_contingency_table)
    a, b = overall_contingency_table[0]
    c, d = overall_contingency_table[1]
    overall_odds_ratio = (a * d) / (b * c) if b * c != 0 else np.nan

    # Process each age group within the filtered dataset
    for age_group, group_data in df_filtered.groupby('age_group'):
        contingency_table = [
            [group_data['CeD_affected'].sum(), total_CeD_not_affected],
            [group_data['healthy_affected'].sum(), total_healthy_not_affected]
        ]
        
        # Compute chi-square test and odds ratio
        chi2, p, dof, ex = chi2_contingency(contingency_table)
        a, b = contingency_table[0]
        c, d = contingency_table[1]
        odds_ratio = (a * d) / (b * c) if b * c != 0 else np.nan
        
        # Store results with formatted p-value
        contingency_tables[age_group] = contingency_table
        chi_square_results[age_group] = {
            "chi2": chi2, 
            "p-value": f"{p:.2e}",  # Format p-value in scientific notation
            "odds_ratio": f"{odds_ratio:.2f}"
        }
    
    # Store all results for the question
    analysis_results[question] = {
        "contingency_tables": contingency_tables,
        "chi_square_results": chi_square_results,
        "overall_p-value": f"{overall_p:.2e}",  # Format overall p-value in scientific notation
        "overall_odds_ratio": f"{overall_odds_ratio:.2f}"
    }

# Save results to a pickle file
with open('analysis_results.pkl', 'wb') as f:
    pickle.dump(analysis_results, f)

# Optional: Save contingency tables and additional metrics to a CSV for easy viewing
contingency_df = pd.DataFrame([
    (q, ag, *ct[0], *ct[1], res['p-value'], res['odds_ratio']) 
    for q, d in analysis_results.items() 
    for ag, ct, res in zip(d['contingency_tables'].keys(), d['contingency_tables'].values(), d['chi_square_results'].values())
], columns=['Question', 'Age Group', 'CeD Affected', 'CeD Not Affected', 'Healthy Affected', 'Healthy Not Affected', 'P-value', 'Odds Ratio'])

# Add overall p-value and odds ratio
overall_stats = pd.DataFrame([
    (q, "Overall", res['overall_p-value'], res['overall_odds_ratio']) 
    for q, res in analysis_results.items()
], columns=['Question', 'Summary', 'Overall P-value', 'Overall Odds Ratio'])

contingency_df.to_csv('contingency_tables.csv', index=False)
overall_stats.to_csv('overall_stats.csv', index=False)


### Contingency with FDR adjustment

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np
import pickle
from statsmodels.stats.multitest import multipletests

# Load the dataset
file_path = '20240612_family history.csv'
df = pd.read_csv(file_path)

# Prepare results storage
analysis_results = {}
all_p_values = []  # List to collect all p-values for FDR correction

# Calculate global totals for affected across all ages for each question
total_affected_per_question = df.groupby('question').agg({
    'CeD_affected': 'sum',
    'healthy_affected': 'sum'
}).to_dict()

# Process each question
for question in df['question'].unique():
    df_filtered = df[df['question'] == question]
    contingency_tables = {}
    chi_square_results = {}
    
    # Total respondents (assuming the same for each age group, adjust if not)
    CeD_respondents_total = df_filtered['CeD_respondent'].iloc[0]
    healthy_respondents_total = df_filtered['healthy_respondants'].iloc[0]

    # Calculate total not affected
    CeD_not_affected_total = CeD_respondents_total - total_affected_per_question['CeD_affected'][question]
    healthy_not_affected_total = healthy_respondents_total - total_affected_per_question['healthy_affected'][question]

    # Process each age group within the filtered dataset
    for age_group, group_data in df_filtered.groupby('age_group'):
        contingency_table = [
            [group_data['CeD_affected'].sum(), CeD_not_affected_total],
            [group_data['healthy_affected'].sum(), healthy_not_affected_total]
        ]
        
        # Compute chi-square test and odds ratio
        chi2, p, dof, ex = chi2_contingency(contingency_table)
        a, b = contingency_table[0]
        c, d = contingency_table[1]
        odds_ratio = (a * d) / (b * c) if b * c != 0 else np.nan
        all_p_values.append(p)  # Collect p-values for FDR adjustment
        
        # Store results
        contingency_tables[age_group] = contingency_table
        chi_square_results[age_group] = {"chi2": chi2, "p-value": p, "odds_ratio": odds_ratio}
    
    # Store all results for the question
    analysis_results[question] = {
        "contingency_tables": contingency_tables,
        "chi_square_results": chi_square_results
    }

# FDR Correction across all tests
_, p_adjusted, _, _ = multipletests(all_p_values, alpha=0.05, method='fdr_bh')

# Update p-values in analysis results with adjusted values
p_index = 0
for question, data in analysis_results.items():
    for age_group in data["chi_square_results"]:
        data["chi_square_results"][age_group]["p-value"] = f"{p_adjusted[p_index]:.2e}"
        p_index += 1

# Save results to a pickle file
with open('analysis_results.pkl', 'wb') as f:
    pickle.dump(analysis_results, f)

# Optional: Save contingency tables and additional metrics to a CSV for easy viewing
contingency_df = pd.DataFrame([
    (q, ag, *ct[0], *ct[1], res['p-value'], f"{res['odds_ratio']:.2f}") 
    for q, d in analysis_results.items() 
    for ag, ct, res in zip(d['contingency_tables'].keys(), d['contingency_tables'].values(), d['chi_square_results'].values())
], columns=['Question', 'Age Group', 'CeD Affected', 'CeD Not Affected', 'Healthy Affected', 'Healthy Not Affected', 'Adjusted P-value', 'Odds Ratio'])

contingency_df.to_csv('contingency_tables.csv', index=False)




### plot the figure

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

# Convert necessary columns to numeric
contingency_df['Adjusted P-value'] = pd.to_numeric(contingency_df['Adjusted P-value'], errors='coerce')
contingency_df['Odds Ratio'] = pd.to_numeric(contingency_df['Odds Ratio'], errors='coerce')

# List of specific health conditions to focus on
specific_conditions = [
    'a skin condition e.g., eczema, psoriasis',
    'acid reflux',
    'an eating disorder',
    'anemia',
    'anxiety reaction/panic disorder',
    'asthma',
    'chronic fatigue',
    "Crohn's disease",
    'fibromyalgia',
    'hyperthyroidism',
    'hypothyroidism',
    'irritable bowel syndrome IBS',
    'Lyme disease',
    'memory loss or impairment',
    'migraine headaches',
    'neuropathy',
    'osteoporosis',
    'reactions to anesthesia such as hyperthermia',
    'systemic lupus',
    'type 1 diabetes',
    'ulcerative colitis',
    'vitamin B deficiency',
    'vitamin D deficiency'
]

# Filter DataFrame for specific questions
contingency_df = contingency_df[contingency_df['Question'].isin(specific_conditions)]

# Create a mask for the required conditions (significantly altered OR and p-value)
criteria_mask = (contingency_df['Adjusted P-value'] < 0.0001) & ((contingency_df['Odds Ratio'] > 1.5) | (contingency_df['Odds Ratio'] < 0.5))
relevant_questions = contingency_df[criteria_mask]['Question'].unique()

# Example data loading and preparation (assuming final_df is loaded correctly)
# Load final_df with 'Question', 'Age Group', 'Adjusted P-value', and 'Odds Ratio' columns prepared
final_df = contingency_df[contingency_df['Question'].isin(relevant_questions)]
final_df = final_df.sort_values(by=['Question', 'Age Group'])

# Define custom age group order and set Categorical type for ordering
age_order = ['Child (0-11)', 'Adolescent (12-17)', 'Adult (18-64)', 'Older adult (65-74)', 'Elderly (75+)']
final_df['Age Group'] = pd.Categorical(final_df['Age Group'], categories=age_order, ordered=True)

# Sort by question and age for consistent plotting
final_df.sort_values(['Question', 'Age Group'], inplace=True)

# Setup a diverging colormap
cmap = plt.get_cmap('coolwarm')

# Normalize OR values for color mapping based on a logical OR range, adjusting here as needed
# Consider realistic OR values; for example, from 0.1 (more protective) to 10 (more risky)
or_min = 0.4  # Modify as per your data range
or_max = 5.0  # Modify as per your data range
norm = mcolors.LogNorm(vmin=or_min, vmax=or_max)  # Log normalization for better visual differentiation

# Plotting
num_questions = len(final_df['Question'].unique())
fig, ax = plt.subplots(figsize=(10, max(2, num_questions * 0.5)))  # Adjust figure size based on the number of questions

# Plot each question as a row with OR values
for i, (question, group) in enumerate(final_df.groupby('Question')):
    y = np.repeat(i, len(group))
    colors = [cmap(norm(or_val)) for or_val in group['Odds Ratio']]
    sizes = np.clip(-np.log10(group['Adjusted P-value']), 0, 40) * 15  # Cap size and scale factor for visibility

    ax.scatter(group['Age Group'], y, s=sizes, color=colors, alpha=0.75)

# Set aesthetics
ax.set_yticks(range(num_questions))
ax.set_yticklabels(final_df['Question'].unique())
ax.set_xlabel('Age Group')
ax.set_title('Heatmap by Age Group for Selected Conditions')
ax.set_ylim(-1, num_questions)
ax.axis('tight')

# Add a legend with colored points representing OR
for size, label in zip(legend_sizes, legend_labels):
    ax.scatter([], [], s=size, color='gray', alpha=0.75, label=label)

# Manually creating a list of OR values for the legend
or_legend_values = [0.5, 1, 2, 3, 4]
or_legend_labels = [f'OR={val}' for val in or_legend_values]
or_legend_colors = [cmap(norm(val)) for val in or_legend_values]
or_legend_sizes = [100] * len(or_legend_values)  # Fixed size for OR legend points

for color, label, size in zip(or_legend_colors, or_legend_labels, or_legend_sizes):
    ax.scatter([], [], s=size, color=color, alpha=0.75, label=label)

# Add legend for Adjusted P-value
# Create dummy scatter points for legend
legend_sizes = [5, 10, 20, 40]  # Example sizes
legend_labels = [f'$10^{{-{x}}}$' for x in legend_sizes]
legend_sizes = [np.clip(-np.log10(10**-x), 0, 40) * 15 for x in legend_sizes]  # Use same scaling as for sizes

for size, label in zip(legend_sizes, legend_labels):
    ax.scatter([], [], s=size, color='gray', alpha=0.75, label=label)

ax.legend(scatterpoints=1, frameon=True, labelspacing=1, title='Adjusted P-value')


plt.savefig('heatmap_by_age_group2.pdf', format='pdf', bbox_inches='tight')

plt.show()


